In [4]:
import torch
torch.cuda.is_available()

True

In [10]:
for x in dir(tensorboard_writer):
    print(x)

__class__
__delattr__
__dict__
__dir__
__doc__
__enter__
__eq__
__exit__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_encode
_get_file_writer
add_audio
add_custom_scalars
add_custom_scalars_marginchart
add_custom_scalars_multilinechart
add_embedding
add_figure
add_graph
add_histogram
add_histogram_raw
add_hparams
add_image
add_image_with_boxes
add_images
add_mesh
add_onnx_graph
add_pr_curve
add_pr_curve_raw
add_scalar
add_scalars
add_tensor
add_text
add_video
all_writers
close
default_bins
file_writer
filename_suffix
flush
flush_secs
get_logdir
log_dir
max_queue
purge_step


In [24]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

batch_size = 100
total_step = 10000

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# MNIST dataset 
dataset = torchvision.datasets.MNIST(
    root='../../data', 
    train=True, 
    transform=transforms.ToTensor(),  
    download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(
    dataset=dataset, 
    batch_size=batch_size, 
    shuffle=True)


# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet().to(device)
tensorboard_writer = SummaryWriter("logs")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  
#from torch.utils.data.dataloader.DataLoader to torch.utils.data.dataloader._DataLoaderIter
data_iter = iter(data_loader) 
iter_per_epoch = len(data_loader)
# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    # Fetch images and labels
    images, labels = next(data_iter) #only when data_loader is converted to data_iter can "next" be used
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    #torch.squeeze: Returns a tensor with all the dimensions of :attr:`input` of size `1` removed.
    #.float() : equals to self.to(torch.float32), if without it:
    # get RuntimeError: Can only calculate the mean of floating types. Got Byte instead.
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 100 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = {
            'loss': loss.item(),
            'accuracy': accuracy.item()
        }

        for tag, value in info.items():
            # scalar_summary(self, tag, value, step): Log a scalar variable.
            tensorboard_writer.add_scalar(tag, value, step+1)



cuda
Step [100/10000], Loss: 2.2062, Acc: 0.37
Step [200/10000], Loss: 2.1217, Acc: 0.67
Step [300/10000], Loss: 2.0138, Acc: 0.73
Step [400/10000], Loss: 1.8891, Acc: 0.77
Step [500/10000], Loss: 1.7238, Acc: 0.72
Step [600/10000], Loss: 1.6126, Acc: 0.78
Step [700/10000], Loss: 1.5203, Acc: 0.79
Step [800/10000], Loss: 1.2958, Acc: 0.85
Step [900/10000], Loss: 1.2720, Acc: 0.82
Step [1000/10000], Loss: 1.1158, Acc: 0.78
Step [1100/10000], Loss: 1.1818, Acc: 0.79
Step [1200/10000], Loss: 1.0087, Acc: 0.83
Step [1300/10000], Loss: 0.9469, Acc: 0.82
Step [1400/10000], Loss: 1.0247, Acc: 0.78
Step [1500/10000], Loss: 0.7590, Acc: 0.92
Step [1600/10000], Loss: 0.7971, Acc: 0.83
Step [1700/10000], Loss: 0.8470, Acc: 0.83
Step [1800/10000], Loss: 0.7787, Acc: 0.83
Step [1900/10000], Loss: 0.7213, Acc: 0.87
Step [2000/10000], Loss: 0.5879, Acc: 0.90
Step [2100/10000], Loss: 0.7075, Acc: 0.86
Step [2200/10000], Loss: 0.5931, Acc: 0.92
Step [2300/10000], Loss: 0.7022, Acc: 0.84
Step [2400/1000